In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install --upgrade google-cloud-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-storage-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.99.0 requires google-cloud-storage<3.0.0,>=1.32.0, but you have google-cloud-storage 3.1.1 which is incompatible.


In [ ]:
from google.cloud import storage

In [ ]:
client = storage.Client(project= 'trip-recommendation-project')
bucket = client.get_bucket('trip-data-cleaning')

In [ ]:
blobs = list(bucket.list_blobs())

In [ ]:
blobs

[<Blob: trip-data-cleaning, destinations.csv, 1751235090758947>,
 <Blob: trip-data-cleaning, members.csv, 1751235091072110>,
 <Blob: trip-data-cleaning, members_clean.csv, 1751235115560472>,
 <Blob: trip-data-cleaning, trips.csv, 1751235090870536>,
 <Blob: trip-data-cleaning, weather.csv, 1751235090709155>]

In [ ]:
csv_files = [blob.name for blob in blobs if blob.name.endswith('.csv')]

In [ ]:
csv_files

['destinations.csv',
 'members.csv',
 'members_clean.csv',
 'trips.csv',
 'weather.csv']

In [ ]:
if not csv_files:
  print("no '.csv' files are found in the bucket!")
else:
  for file in csv_files:
    blob = bucket.blob(file)
    blob.download_to_filename(file)
    print(f"Downloaded: {file}")

Downloaded: destinations.csv
Downloaded: members.csv
Downloaded: members_clean.csv
Downloaded: trips.csv
Downloaded: weather.csv


# Members.CSV File Cleaning

In [ ]:
# cleaning members.csv first
import numpy as np
import pandas as pd

df_members = pd.read_csv('members.csv')

In [ ]:
df_members.shape

(1005, 9)

In [ ]:
df_members.head()

,member_id,name,age,location,travel_budget,signup_date,preference,num_dependents,has_passport
0,1685,michael williams,71,Stevenstad- IA,646000,17/07/2021,luxury,3,False
1,1614,Danielle george,50,"Jessicabury, NC",153000,2024-04-06,adventure,0,True
2,1602,denise brock,22,Salinasfort- ND,155000,2024-09-23,beach,3,True
3,1502,Terry holmes,unknown,South Megan- MP,403000,"Apr 24, 2024",beach,3,True
4,1341,michelle walker,25,"Davidburgh, CO",693000,22/01/2023,cultural,3,True


### Cleaning Plan for `members.csv` (To-Do)

Before using the `members.csv` data for analysis or joining, I need to perform the following cleaning steps:

| Column           | Cleaning To-Do                                                                 |
|------------------|----------------------------------------------------------------------------------|
| `member_id`      | Convert to integer, ensure values are unique and non-null.                      |
| `name`           | Strip whitespace and apply `.title()` to standardize casing.                    |
| `age`            | Convert to numeric and filter valid range (18–100).                             |
| `location`       | Remove non-alphabetic characters and standardize to `"City, ST"` format.        |
| `travel_budget`  | Ensure numeric, drop missing or extreme values (outside $50k–$1M).              |
| `signup_date`    | Parse mixed formats to `YYYY-MM-DD`, drop rows with invalid dates.              |
| `preference`     | Normalize to lowercase and keep only valid values like adventure, beach, etc.   |
| `num_dependents` | Convert to integer, fill NAs with 0, and drop negatives.                        |
| `has_passport`   | Map text to boolean (`True`/`False`), drop rows with invalid values.            |

I'll apply each of these in Pandas using `.astype()`, `.str.strip()`, `.to_numeric()`, `.to_datetime()`, `.map()`, and filtering functions.


In [ ]:
# code

# name variable cleaning

df_members['name'].dtype

dtype('O')

## Name Variable Cleaning

In [ ]:
df_members['name'] = df_members['name'].astype(str).str.strip().str.title()

In [ ]:
type(df_members['name'].iloc[0])

str

## Age Variable Cleaning

In [ ]:
# age variable cleaning

type(df_members['age'].iloc[0])

str

In [ ]:
df_members['age'].unique()

array(['71', '50', '22', 'unknown', '25', '34', '68', '33', '74', '54',
       '-3', nan, '55', '57', '56', '49', '75', '27', '63', '62', '39',
       '44', '70', '67', '21', '36', '77', '58', '61', '48', '20', '66',
       '28', '24', '76', '64', '45', '30', '52', '41', '42', '47', '43',
       '78', '65', '80', '46', '18', '59', '23', '73', '53', '60', '26',
       '72', '38', '37', '29', '31', '51', '69', '32', '40', '35', '19',
       '79'], dtype=object)

In [ ]:
df_members['age'] = df_members['age'].replace(['','unknown','-3'], np.nan)

In [ ]:
df_members['age'].value_counts(dropna=False)

,count
age,
NaN,95
50,23
43,22
76,21
64,21
...,...
31,10
62,9
72,9


In [ ]:
df_members['age'] = pd.to_numeric(df_members['age']) # converting str to int

In [ ]:
# imputing the missing values with median of the particular preference group to look more realistic

df_members['age'] = df_members.groupby('preference')['age'].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
df_members['age'].value_counts(dropna=False)

,count
age,
48.0,48
48.5,38
50.0,38
43.0,22
64.0,21
...,...
31.0,10
62.0,9
72.0,9


In [ ]:
df_members['age'].isnull().sum()

np.int64(0)

## Location Variable Cleaning

In [ ]:
df_members['location'].head(5)

,location
0,Stevenstad- IA
1,"Jessicabury, NC"
2,Salinasfort- ND
3,South Megan- MP
4,"Davidburgh, CO"


In [ ]:
df_members['location'] = df_members['location'].str.replace(r'[\-/]', ',', regex=True)

In [ ]:
df_members['location'].head(5)

,location
0,"Stevenstad, IA"
1,"Jessicabury, NC"
2,"Salinasfort, ND"
3,"South Megan, MP"
4,"Davidburgh, CO"


In [ ]:
df_members[['city', 'state']] = df_members['location'].str.extract(r'^\s*([\w\s]+)\s*,\s*([A-Z]{2})\s*$')

In [ ]:
df_members.head(3)

,member_id,name,age,location,travel_budget,signup_date,preference,num_dependents,has_passport,city,state
0,1685,Michael Williams,71.0,"Stevenstad, IA",646000,17/07/2021,luxury,3,False,Stevenstad,IA
1,1614,Danielle George,50.0,"Jessicabury, NC",153000,2024-04-06,adventure,0,True,Jessicabury,NC
2,1602,Denise Brock,22.0,"Salinasfort, ND",155000,2024-09-23,beach,3,True,Salinasfort,ND


In [ ]:
df_members.drop(columns = 'location', inplace = True)

In [ ]:
df_members['city'] = df_members['city'].str.strip().str.title()
df_members['state'] = df_members['state'].str.strip().str.upper()

In [ ]:
df_members.head(5)

,member_id,name,age,travel_budget,signup_date,preference,num_dependents,has_passport,city,state
0,1685,Michael Williams,71.0,646000,17/07/2021,luxury,3,False,Stevenstad,IA
1,1614,Danielle George,50.0,153000,2024-04-06,adventure,0,True,Jessicabury,NC
2,1602,Denise Brock,22.0,155000,2024-09-23,beach,3,True,Salinasfort,ND
3,1502,Terry Holmes,50.0,403000,"Apr 24, 2024",beach,3,True,South Megan,MP
4,1341,Michelle Walker,25.0,693000,22/01/2023,cultural,3,True,Davidburgh,CO


## Travel_budget feature cleaning

In [ ]:
df_members['travel_budget'].value_counts(dropna=False).head(5)


,count
travel_budget,
115000,6
571000,5
224000,5
474000,5
648000,5


In [ ]:
df_members['travel_budget'].isna().sum()

np.int64(0)

In [ ]:
non_numeric = df_members[~df_members['travel_budget'].astype(str).str.replace(',', '').str.replace('$', '').str.strip().str.isnumeric()]
non_numeric[['travel_budget']]

,travel_budget


## Signup_date feature cleaning

In [ ]:
df_members['signup_date'].head(10)

,signup_date
0,17/07/2021
1,2024-04-06
2,2024-09-23
3,"Apr 24, 2024"
4,22/01/2023
5,2022-09-13
6,"Sep 11, 2024"
7,10/01/2025
8,"Sep 09, 2021"
9,"Nov 19, 2022"


In [ ]:
df_members['signup_date'].apply(lambda x: str(x)[:4]).value_counts()


,count
signup_date,
2023,85
2022,84
2024,76
2021,41
Aug,39
...,...
25/1,1
30/1,1
19/1,1


In [ ]:
df_members['signup_date'].value_counts(dropna=False)

,count
signup_date,
12/06/2024,4
2023-01-08,3
03/12/2021,3
"Aug 28, 2022",3
2022-11-20,3
...,...
2023-08-04,1
30/04/2023,1
"Mar 12, 2023",1


In [ ]:
df = df_members.copy()

In [ ]:
df['signup_date'] = df['signup_date'].astype(str).str.strip()

In [ ]:
df['signup_date_clean'] = pd.NaT

In [ ]:
mask_iso = df['signup_date'].str.match(r'^\d{4}-\d{2}-\d{2}$')
df.loc[mask_iso, 'signup_date_clean'] = pd.to_datetime(df.loc[mask_iso, 'signup_date'], format='%Y-%m-%d')

In [ ]:
mask_dmy = df['signup_date'].str.match(r'^\d{2}/\d{2}/\d{4}$')
df.loc[mask_dmy, 'signup_date_clean'] = pd.to_datetime(df.loc[mask_dmy, 'signup_date'], format='%d/%m/%Y')

In [ ]:
mask_named_month = df['signup_date'].str.match(r'^[A-Za-z]{3,9} \d{1,2}, \d{4}$')
df.loc[mask_named_month, 'signup_date_clean'] = pd.to_datetime(df.loc[mask_named_month, 'signup_date'], format='%b %d, %Y')

In [ ]:
df['signup_date_clean'] = df['signup_date_clean'].dt.strftime('%Y-%m-%d')

In [ ]:
df['signup_date_clean'].head(10)

,signup_date_clean
0,2021-07-17
1,2024-04-06
2,2024-09-23
3,2024-04-24
4,2023-01-22
5,2022-09-13
6,2024-09-11
7,2025-01-10
8,2021-09-09
9,2022-11-19


In [ ]:
df['signup_date_clean'].isna().sum()

np.int64(0)

In [ ]:
df.head(4)

,member_id,name,age,travel_budget,signup_date,preference,num_dependents,has_passport,city,state,signup_date_clean
0,1685,Michael Williams,71.0,646000,17/07/2021,luxury,3,False,Stevenstad,IA,2021-07-17
1,1614,Danielle George,50.0,153000,2024-04-06,adventure,0,True,Jessicabury,NC,2024-04-06
2,1602,Denise Brock,22.0,155000,2024-09-23,beach,3,True,Salinasfort,ND,2024-09-23
3,1502,Terry Holmes,50.0,403000,"Apr 24, 2024",beach,3,True,South Megan,MP,2024-04-24


In [ ]:
df.drop(columns='signup_date', inplace=True)

In [ ]:
df.head(5)

,member_id,name,age,travel_budget,preference,num_dependents,has_passport,city,state,signup_date_clean
0,1685,Michael Williams,71.0,646000,luxury,3,False,Stevenstad,IA,2021-07-17
1,1614,Danielle George,50.0,153000,adventure,0,True,Jessicabury,NC,2024-04-06
2,1602,Denise Brock,22.0,155000,beach,3,True,Salinasfort,ND,2024-09-23
3,1502,Terry Holmes,50.0,403000,beach,3,True,South Megan,MP,2024-04-24
4,1341,Michelle Walker,25.0,693000,cultural,3,True,Davidburgh,CO,2023-01-22


## Preference feature cleaning

In [ ]:
df['preference'].unique()

array(['luxury', 'adventure', 'beach', 'cultural', 'nature', 'budget'],
      dtype=object)

In [ ]:
# no cleaning required

## Num of Dependents

In [ ]:
df['num_dependents'].unique()

array([3, 0, 1, 2])

In [ ]:
type(df['num_dependents'][0])

numpy.int64

## Has_passport?

In [ ]:
df['has_passport'].unique()

array([False,  True])

In [ ]:
df[df['has_passport'].isnull()]

,member_id,name,age,travel_budget,preference,num_dependents,has_passport,city,state,signup_date_clean


In [ ]:
df.head()

,member_id,name,age,travel_budget,preference,num_dependents,has_passport,city,state,signup_date_clean
0,1685,Michael Williams,71.0,646000,luxury,3,False,Stevenstad,IA,2021-07-17
1,1614,Danielle George,50.0,153000,adventure,0,True,Jessicabury,NC,2024-04-06
2,1602,Denise Brock,22.0,155000,beach,3,True,Salinasfort,ND,2024-09-23
3,1502,Terry Holmes,50.0,403000,beach,3,True,South Megan,MP,2024-04-24
4,1341,Michelle Walker,25.0,693000,cultural,3,True,Davidburgh,CO,2023-01-22


## Check for Duplicates

In [ ]:
df[df.duplicated()]

,member_id,name,age,travel_budget,preference,num_dependents,has_passport,city,state,signup_date_clean
633,1818,Vanessa Stephenson,64.0,130000,beach,1,True,Arnoldberg,DC,2022-06-03
670,1242,Thomas Hart,34.0,561000,beach,3,True,Port Clifford,AZ,2022-04-02
709,1507,Lawrence Robles,65.0,372000,beach,1,True,Sandersville,MT,2023-01-18
728,1452,Laura Jackson,48.5,177000,cultural,2,True,Jenniferbury,AZ,2024-06-12
740,1368,Jerry Carter,28.0,571000,beach,2,False,Hollyborough,MO,2023-10-01


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df[df.duplicated()]

,member_id,name,age,travel_budget,preference,num_dependents,has_passport,city,state,signup_date_clean


In [ ]:
df.to_csv('members_clean.csv', index = False)

In [ ]:
bucket_name = "trip-data-cleaning"

client = storage.Client()
bucket = client.bucket(bucket_name)

blob = bucket.blob('members_clean.csv')
blob.upload_from_filename('members_clean.csv')

print("Uploaded Successfully!")

Uploaded Successfully!


# Destinations.csv File Cleaning

In [ ]:
df1 = pd.read_csv('destinations.csv')

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   destination    10 non-null     object 
 1   country        10 non-null     object 
 2   tags           10 non-null     object 
 3   avg_cost_usd   10 non-null     int64  
 4   safety_rating  10 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 532.0+ bytes


In [ ]:
# Looking into info there are no missing values.

In [ ]:
df1.isna().sum()

,0
destination,0
country,0
tags,0
avg_cost_usd,0
safety_rating,0


In [ ]:
df1['tags'].unique()

array(['budget-friendly,romantic,cultural,historic', 'luxury,beach',
       'historic,party,beach,budget-friendly',
       'budget-friendly,romantic,historic', 'cultural,nature,adventure',
       'beach,budget-friendly,historic,party', 'historic,family-friendly',
       'luxury,historic,romantic',
       'historic,party,family-friendly,adventure',
       'romantic,family-friendly,budget-friendly'], dtype=object)

## Tags Feature cleaning

In [ ]:
df1['tags'] = df1['tags'].astype(str).str.split(',')

In [ ]:
df1.head()

,destination,country,tags,avg_cost_usd,safety_rating
0,Cancun,Mexico,"[budget-friendly, romantic, cultural, historic]",80000,4.6
1,Paris,France,"[luxury, beach]",502000,3.1
2,Tokyo,Japan,"[historic, party, beach, budget-friendly]",101000,3.9
3,Bali,Indonesia,"[budget-friendly, romantic, historic]",109000,2.5
4,Cape Town,South Africa,"[cultural, nature, adventure]",279000,4.8


In [ ]:
df1['tags'] = df1['tags'].apply(
    lambda lst: [tag.strip().lower() for tag in lst]
)

In [ ]:
df1.head()

,destination,country,tags,avg_cost_usd,safety_rating
0,Cancun,Mexico,"[budget-friendly, romantic, cultural, historic]",80000,4.6
1,Paris,France,"[luxury, beach]",502000,3.1
2,Tokyo,Japan,"[historic, party, beach, budget-friendly]",101000,3.9
3,Bali,Indonesia,"[budget-friendly, romantic, historic]",109000,2.5
4,Cape Town,South Africa,"[cultural, nature, adventure]",279000,4.8


In [ ]:
df1.to_csv('destinations_clean.csv', index = False)

In [ ]:
bucket_name = "trip-data-cleaning"

client = storage.Client()
bucket = client.bucket(bucket_name)

blob = bucket.blob('destinations_clean.csv')
blob.upload_from_filename('destinations_clean.csv')

print("Uploaded Successfully!")

Uploaded Successfully!


# Trips.csv File Cleaning

In [ ]:
df2 = pd.read_csv('trips.csv')

In [ ]:
df2.head()

,trip_id,member_id,destination,start_date,duration_days,cost_usd,activities
0,T1000,1685,Cape Town,2023-04-12,5,286000,"wine tasting, shopping"
1,T1001,1685,Banff,"January 15, 2025",10,259000,"shopping, wine tasting"
2,T1002,1685,Banff,11/18/2024,8,268000,"shopping, wine tasting, spa"
3,T1003,1685,Barcelona,04/18/2025,9,176000,"wine tasting, shopping, spa"
4,T1004,1685,Rome,2025-01-29,6,232000,"shopping, wine tasting"


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3011 entries, 0 to 3010
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   trip_id        3011 non-null   object
 1   member_id      3011 non-null   int64 
 2   destination    3011 non-null   object
 3   start_date     3011 non-null   object
 4   duration_days  3011 non-null   int64 
 5   cost_usd       3011 non-null   int64 
 6   activities     3011 non-null   object
dtypes: int64(3), object(4)
memory usage: 164.8+ KB


In [ ]:
df2.isna().sum()

,0
trip_id,0
member_id,0
destination,0
start_date,0
duration_days,0
cost_usd,0
activities,0


## Trip id Feature cleaning

In [ ]:
df2[df2['trip_id'].duplicated()] # check if any duplicate primary keys

,trip_id,member_id,destination,start_date,duration_days,cost_usd,activities


## Destination

In [ ]:
df2['destination'].unique()

array(['Cape Town', 'Banff', 'Barcelona', 'Rome', 'Bali', 'Tokyo',
       'Cancun', 'Denver', 'Phuket', 'Paris'], dtype=object)

In [ ]:
type(df2['destination'][0])

str

In [ ]:
df2['destination'] = df2['destination'].astype(str).str.strip().str.title()

## Start_date feature cleaning

In [ ]:
df2['start_date_clean'] = pd.NaT

In [ ]:
df2.head(2)

,trip_id,member_id,destination,start_date,duration_days,cost_usd,activities,start_date_clean
0,T1000,1685,Cape Town,2023-04-12,5,286000,"wine tasting, shopping",NaT
1,T1001,1685,Banff,"January 15, 2025",10,259000,"shopping, wine tasting",NaT


In [ ]:
mask_iso = df2['start_date'].str.match(r'^\d{4}-\d{2}-\d{2}$')
mask_dmy = df2['start_date'].str.match(r'^\d{2}/\d{2}/\d{4}$')
mask_monthname = df2['start_date'].str.match(r'^[A-Za-z]{3,9} \d{1,2}, \d{4}$')

In [ ]:
df2.loc[mask_iso, 'start_date_clean'] = pd.to_datetime(df2.loc[mask_iso, 'start_date'], format='%Y-%m-%d')
df2.loc[mask_dmy, 'start_date_clean'] = pd.to_datetime(df2.loc[mask_dmy, 'start_date'], format='%m/%d/%Y')
df2.loc[mask_monthname, 'start_date_clean'] = pd.to_datetime(df2.loc[mask_monthname, 'start_date'], format='%B %d, %Y')

In [ ]:
df2['start_date_clean'] = df2['start_date_clean'].dt.strftime('%Y-%m-%d')

In [ ]:
df2.head()

,trip_id,member_id,destination,start_date,duration_days,cost_usd,activities,start_date_clean
0,T1000,1685,Cape Town,2023-04-12,5,286000,"wine tasting, shopping",2023-04-12
1,T1001,1685,Banff,"January 15, 2025",10,259000,"shopping, wine tasting",2025-01-15
2,T1002,1685,Banff,11/18/2024,8,268000,"shopping, wine tasting, spa",2024-11-18
3,T1003,1685,Barcelona,04/18/2025,9,176000,"wine tasting, shopping, spa",2025-04-18
4,T1004,1685,Rome,2025-01-29,6,232000,"shopping, wine tasting",2025-01-29


In [ ]:
df2.drop(columns='start_date', inplace=True)

## Duration_days

In [ ]:
df2[df2['duration_days'] <= 0]

,trip_id,member_id,destination,duration_days,cost_usd,activities,start_date_clean


## Cost_usd

In [ ]:
df2['cost_usd'].min()

64000

In [ ]:
df2['cost_usd'].max()

615000

In [ ]:
df2.loc[df2['cost_usd'] == df2['cost_usd'].min()]

,trip_id,member_id,destination,duration_days,cost_usd,activities,start_date_clean
297,T1297,1154,Cancun,3,64000,"spa, shopping",2022-10-21
522,T1522,1354,Cancun,8,64000,"spa, shopping",2023-12-17
917,T1917,1757,Cancun,5,64000,"food tour, museum tour, cultural show",2022-04-06
2392,T3392,1450,Cancun,7,64000,"kayaking, hiking, safari",2025-03-23
2617,T3617,1712,Cancun,7,64000,"spa, beach",2022-09-17


## Activities

In [ ]:
df2['activities'].unique()

array(['wine tasting, shopping', 'shopping, wine tasting',
       'shopping, wine tasting, spa', 'wine tasting, shopping, spa',
       'kayaking, hiking', 'hiking, safari, kayaking',
       'kayaking, hiking, safari', 'hiking, safari',
       'beach, snorkeling, spa', 'snorkeling, beach', 'beach',
       'snorkeling, spa, beach', 'spa, beach, snorkeling',
       'cultural show, food tour, museum tour', 'snorkeling, spa',
       'safari', 'spa', 'spa, beach', 'hiking', 'safari, kayaking',
       'safari, hiking', 'kayaking', 'safari, hiking, kayaking',
       'spa, snorkeling', 'beach, spa, snorkeling', 'wine tasting, spa',
       'shopping, spa, wine tasting', 'hiking, museum tour', 'food tour',
       'hiking, museum tour, food tour', 'hiking, kayaking',
       'food tour, cultural show, museum tour',
       'food tour, cultural show',
       'museum tour, cultural show, food tour',
       'cultural show, food tour', 'snorkeling, beach, spa', 'snorkeling',
       'beach, spa', 'beach,

In [ ]:
df2['activities'] = df2['activities'].str.split(',')

In [ ]:
df2['activities'] = df2['activities'].apply(
    lambda lst: [activity.strip().lower() for activity in lst]
)

In [ ]:
df2.head()

,trip_id,member_id,destination,duration_days,cost_usd,activities,start_date_clean
0,T1000,1685,Cape Town,5,286000,"[wine tasting, shopping]",2023-04-12
1,T1001,1685,Banff,10,259000,"[shopping, wine tasting]",2025-01-15
2,T1002,1685,Banff,8,268000,"[shopping, wine tasting, spa]",2024-11-18
3,T1003,1685,Barcelona,9,176000,"[wine tasting, shopping, spa]",2025-04-18
4,T1004,1685,Rome,6,232000,"[shopping, wine tasting]",2025-01-29


In [ ]:
df2.to_csv('trips_clean.csv', index=False)

In [ ]:
bucket_name = "trip-data-cleaning"

client = storage.Client()
bucket = client.bucket(bucket_name)

blob = bucket.blob('trips_clean.csv')
blob.upload_from_filename('trips_clean.csv')

print("Uploaded Successfully!")

Uploaded Successfully!


# Weather.csv Cleaning

In [ ]:
df3 = pd.read_csv('weather.csv')

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   destination      120 non-null    object 
 1   month            120 non-null    object 
 2   weather          120 non-null    object 
 3   seasonal_rating  120 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.9+ KB


In [ ]:
df3.isna().sum()

,0
destination,0
month,0
weather,0
seasonal_rating,0


In [ ]:
df3.head()

,destination,month,weather,seasonal_rating
0,Cancun,January,stormy,4.1
1,Cancun,February,rainy,3.1
2,Cancun,March,cloudy,4.1
3,Cancun,April,rainy,4.6
4,Cancun,May,cloudy,4.8


## Destination

In [ ]:
df3['destination'].unique()

array(['Cancun', 'Paris', 'Tokyo', 'Bali', 'Cape Town', 'Denver', 'Banff',
       'Phuket', 'Rome', 'Barcelona'], dtype=object)

## Month

In [ ]:
df3['month'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

## Weather

In [ ]:
df3['weather'].unique()

array(['stormy', 'rainy', 'cloudy', 'snowy', 'windy', 'sunny'],
      dtype=object)

## Seasonal_rating

In [ ]:
df3['seasonal_rating'].isnull().sum()

np.int64(0)

In [ ]:
df3['seasonal_rating'].min()

2.5

In [ ]:
df3['seasonal_rating'].max()

5.0

In [ ]:
df3.to_csv('weather_clean.csv', index = False)

In [ ]:
blob = bucket.blob('weather_clean.csv')
blob.upload_from_filename('weather_clean.csv')

print("Uploaded Successfully!")

Uploaded Successfully!
